# Common DataFrame manipulations to all storage options 

In [1]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DATA_DIR = os.path.join(DATA_DIR, 'interim')

INPUT_LOAD_AND_VRE_FILENAME = 'load_and_vre.csv'
FINAL_LOAD_AND_VRE_FILENAME = 'final_load_and_vre.csv'

In [2]:
# all input columns as variables for later use
date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'

load_and_vre_df = pd.read_csv(os.path.join(RAW_DATA_DIR, INPUT_LOAD_AND_VRE_FILENAME))
input_cols = [date_colname, load_colname, wind_gen_colname, solar_gen_colname]
load_and_vre_df = load_and_vre_df.iloc[:, [i for i in range(len(input_cols))]]
load_and_vre_df.columns = input_cols

In [21]:
# all derived columns as variables for later use

### COMMON - columns needed bt legacy gen, batteries, hydrogen
# hourly 
total_vre_gen_colname = 'total_vre_mw'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'
critical_load_mw_colname = 'critical_load_mw'
critical_load_less_vre_colname = 'critical_load_less_vre_mw'
surplus_vre_colname = 'surplus_vre'

# daily
daily_load_vre_diff_colname = 'daily_load_vre_diff'
prev_7_load_colname = 'prev_7_load'
prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
prev_7_vre_gen_colname = 'prev_7_vre_gen'

### BATTERIES 
cuml_load_since_prev_charge_colname = 'cuml_load_since_prev_charge_mw'
cuml_charge_since_prev_discharge_colname = 'cuml_charge_since_prev_discharge_mw'
battery_soc_colname = 'battery_soc'

### HYDROGEN
req_hydrogen_kg_colname = 'reqd_hydrogen_kg'
hydrogen_production_kg_colname = 'hydrogen_prod_kg'
hydrogen_production_m3_colname = 'hydrogen_prod_m3'
hydrogen_h2o_demand_l_colname = 'hydrogen_h2o_demand_l'
hydrogen_h2o_demand_m3_colname = 'hydrogen_h2o_demand_m3'
compression_desal_demand_mw_colname = 'compression_desal_demand_mw' 
cuml_hydrogen_demand_kg_colname = 'cuml_hydrogen_demand_kg'
cuml_h2_energy_demand_mwh_colname = 'cuml_h2_energy_demand_mwh'

### BAU - LEGACY GENERATION
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_rice_mw_colname = 'thermal_gen_rice_mw'
thermal_gen_combustion_mw_colname = 'thermal_gen_combustion_mw'

In [4]:
# other values/assumptions needed for the analysis
CRITICAL_LOAD_PERC = (1/3)
SOLAR_SCALE_FACTOR = 1
WIND_SCALE_FACTOR = 1

SOLAR_INSTALLATION_COST_MM = 50.8
WIND_INSTALLATION_COST_MM = 31.2

In [5]:
def clean_input_columns(df):

    cleaned_df = df.copy()
    cleaned_df = cleaned_df.dropna(how='all')

    cleaned_df[date_colname] = pd.to_datetime(cleaned_df[date_colname])
    # all solar vals need to be > 0
    cleaned_df[solar_gen_colname] = cleaned_df[solar_gen_colname]\
        .apply(lambda x: x if x > 0 else 0)
    
    cleaned_df[month_colname] = cleaned_df[date_colname].map(lambda x: x.month)
    cleaned_df[day_of_month_colname] = cleaned_df[date_colname].map(lambda x: x.day)
    cleaned_df[day_of_yr_colname] = cleaned_df[date_colname].map(lambda x: x.timetuple().tm_yday)
    
    return cleaned_df

In [6]:
cleaned_load_and_vre_df = clean_input_columns(load_and_vre_df)

In [7]:
assert len(cleaned_load_and_vre_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

for day_of_yr in cleaned_load_and_vre_df[day_of_yr_colname].unique():
    if len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr]) != 24:
        assert ValueError("Day {} has {} values".format(day_of_yr, len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr])))
        

In [8]:
def add_common_derived_columns(df):
    
    decorated_df = df.copy()
    
    decorated_df[solar_gen_colname] = decorated_df[solar_gen_colname] * SOLAR_SCALE_FACTOR
    decorated_df[wind_gen_colname] = decorated_df[wind_gen_colname] * WIND_SCALE_FACTOR
    
    decorated_df[total_vre_gen_colname] = \
        decorated_df[wind_gen_colname] + decorated_df[solar_gen_colname]
    
    decorated_df[critical_load_mw_colname] = \
        decorated_df[load_colname] * CRITICAL_LOAD_PERC
    
    decorated_df[critical_load_less_vre_colname] = \
        decorated_df[critical_load_mw_colname] - decorated_df[total_vre_gen_colname]
    
    decorated_df[surplus_vre_colname] = \
        -decorated_df[critical_load_less_vre_colname]

    decorated_df[critical_load_less_vre_colname] = decorated_df[critical_load_less_vre_colname]\
        .map(lambda x: x if x > 0 else 0)

    decorated_df[surplus_vre_colname] = decorated_df[surplus_vre_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    return decorated_df

In [9]:
final_load_and_vre_df = add_common_derived_columns(cleaned_load_and_vre_df)
final_load_and_vre_df.to_csv(os.path.join(INTERIM_DATA_DIR, FINAL_LOAD_AND_VRE_FILENAME), index=False)

# Hydrogen-specific analysis

In [16]:
ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG = 19.988333333333
ELEC_EFF_ELECTROLYSIS_KWH_PER_KG = 55.4307635

ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1.8
DENSITY_H2_300BAR_KG_PER_M3 = 20
DENSITY_H2_30BAR_KG_PER_M3 = 2.38
H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3 = 1.4

DENSITY_H2_STP_KG_PER_M3 = 0.0813
H2_TAP_WATER_CONSUMPTION_L_PER_M3 = 1.4

ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1
ENERGY_CONSUMPTION_H2_KWH_PER_M3 = 1

def calc_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[req_hydrogen_kg_colname] = \
        (decorated_df[critical_load_less_vre_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df[hydrogen_production_kg_colname] = \
        (decorated_df[surplus_vre_colname] * 1000) / ELEC_EFF_ELECTROLYSIS_KWH_PER_KG
    
    decorated_df[hydrogen_production_m3_colname] = \
        decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_30BAR_KG_PER_M3
    
    decorated_df[hydrogen_h2o_demand_l_colname] = \
        decorated_df[hydrogen_production_m3_colname] * H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3
    
    decorated_df[hydrogen_h2o_demand_m3_colname] = \
        (decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_STP_KG_PER_M3) \
        * (H2_TAP_WATER_CONSUMPTION_L_PER_M3 / 1000)
    
#     decorated_df[compression_desal_demand_mw_colname] = \
#         (decorated_df[hydrogen_production_kg_colname] * ENERGY_CONSUMPTION_H2_KWH_PER_M3) \
#         + ()
    decorated_df.sort_values(date_colname)
    
## calc cuml h2 demand
    decorated_df[cuml_hydrogen_demand_kg_colname] = None
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = row[req_hydrogen_kg_colname]
        else : #
            val = \
                decorated_df.loc[idx - 1, cuml_hydrogen_demand_kg_colname] \
                + row[req_hydrogen_kg_colname]\
                - row[hydrogen_production_kg_colname]
        decorated_df.at[idx, cuml_hydrogen_demand_kg_colname] = val
        
    decorated_df[cuml_h2_energy_demand_mwh_colname] = \
        (decorated_df[cuml_hydrogen_demand_kg_colname] * ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG) / 1000
    
    return decorated_df

In [19]:
h2_hourly_df = calc_h2_cols(final_load_and_vre_df)

h2_daily_df = h2_hourly_df.groupby(day_of_yr_colname).sum().reset_index()
h2_daily_df = h2_daily_df.drop([month_colname, day_of_month_colname], axis=1)

In [20]:
h2_hourly_df[
    (h2_hourly_df[month_colname] == 8) & 
    (h2_hourly_df[day_of_month_colname] == 14)]

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,surplus_vre,reqd_hydrogen_kg,hydrogen_prod_kg,hydrogen_prod_m3,hydrogen_h2o_demand_l,hydrogen_h2o_demand_m3,cuml_hydrogen_demand_kg,cuml_h2_energy_demand_mwh
5400,2017-08-14 00:00:00,40.8,3.39149,0.000,8,14,226,3.39149,13.600000,10.208510,0.000000,510.723422,0.000000,0.000000,0.000000,0.000000,1.02197e+06,20427.5
5401,2017-08-14 01:00:00,39.2,4.36958,0.000,8,14,226,4.36958,13.066667,8.697087,0.000000,435.108146,0.000000,0.000000,0.000000,0.000000,1.02241e+06,20436.2
5402,2017-08-14 02:00:00,38.3,4.73765,0.000,8,14,226,4.73765,12.766667,8.029017,0.000000,401.685150,0.000000,0.000000,0.000000,0.000000,1.02281e+06,20444.3
5403,2017-08-14 03:00:00,37.5,5.02921,0.000,8,14,226,5.02921,12.500000,7.470790,0.000000,373.757525,0.000000,0.000000,0.000000,0.000000,1.02318e+06,20451.7
5404,2017-08-14 04:00:00,37.5,4.01208,0.000,8,14,226,4.01208,12.500000,8.487920,0.000000,424.643709,0.000000,0.000000,0.000000,0.000000,1.02361e+06,20460.2
5405,2017-08-14 05:00:00,37.6,4.80651,0.000,8,14,226,4.80651,12.533333,7.726823,0.000000,386.566664,0.000000,0.000000,0.000000,0.000000,1.024e+06,20468
5406,2017-08-14 06:00:00,38.5,4.45444,0.000,8,14,226,4.45444,12.833333,8.378893,0.000000,419.189194,0.000000,0.000000,0.000000,0.000000,1.02441e+06,20476.3
5407,2017-08-14 07:00:00,40.3,3.06793,0.411,8,14,226,3.47893,13.433333,9.954403,0.000000,498.010673,0.000000,0.000000,0.000000,0.000000,1.02491e+06,20486.3
5408,2017-08-14 08:00:00,43.5,4.70032,4.017,8,14,226,8.71732,14.500000,5.782680,0.000000,289.302760,0.000000,0.000000,0.000000,0.000000,1.0252e+06,20492.1
5409,2017-08-14 09:00:00,44.2,4.59258,9.557,8,14,226,14.14958,14.733333,0.583753,0.000000,29.204703,0.000000,0.000000,0.000000,0.000000,1.02523e+06,20492.7


# Batteries